In [0]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import spacy
from spacy.lang.en import English
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [0]:
!pip install spacy
#!apt install -qq enchant
!pip install pyenchant
!python -m spacy download en


Usage:   
  pip <command> [options]

no such option: -m


In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/train.csv',encoding="utf-8")

In [0]:
y=data['target']
X=data['text']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14)

In [0]:
Text_train=X_train.values.tolist()
Text_test=X_test.values.tolist()

In [0]:
#Tokenizer found in the DMML class lab 6.1
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [0]:
Clean_train=[]
for i in Text_train:
  Clean_train.append(spacy_tokenizer(i))

Train_Clean=[]
for i in range(0, len(Clean_train)):
  Train_Clean.append(' '.join(Clean_train[i]))


Clean_test=[]
for i in Text_test:
  Clean_test.append(spacy_tokenizer(i))

Test_Clean=[]
for i in range(0, len(Clean_test)):
  Test_Clean.append(' '.join(Clean_test[i]))



In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [0]:
count = CountVectorizer()
count2=CountVectorizer(ngram_range=(1, 2))
bow_train = count.fit_transform(Train_Clean)
bow_train2=count2.fit_transform(Train_Clean)
bow_train.toarray()
bow_train2.toarray()
feature_names = count.get_feature_names()

In [0]:
tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(Train_Clean)
tfidf_test=tfidf.transform(Test_Clean)

In [0]:
bow_test = count.transform(Test_Clean)
bow_test2 = count2.transform(Test_Clean)
bow_test.toarray()
bow_test2.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
RF = RandomForestClassifier(criterion="entropy", n_estimators=36, max_depth=172, random_state=8)

In [0]:
RF.fit(bow_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=172, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=36,
                       n_jobs=None, oob_score=False, random_state=8, verbose=0,
                       warm_start=False)

In [0]:
print(RF.score(bow_test,y_test))

0.8056467498358503


In [0]:
xgc = xgb.XGBClassifier(n_estimators=101, max_depth=71, base_score=0.5, objective='binary:logistic', random_state=42)
xgc.fit(bow_train,y_train)
print(xgc.score(bow_test,y_test))

In [0]:
0.7957977675640184   101   71

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [0]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf',MultinomialNB()),])

In [0]:
text_clf.fit(Text_train, y_train)
print(text_clf.score(Text_test,y_test))

0.8115561391989494


In [0]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import layers

In [0]:
input_dim = bow_train.shape[1] 
model = Sequential()
model.add(layers.Dense(600, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(200))
model.add(layers.Dense(150))
model.add(layers.Dense(10))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 512)               9479680   
_________________________________________________________________
dense_49 (Dense)             (None, 50)                25650     
_________________________________________________________________
dense_50 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_51 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 21        
Total params: 9,507,501
Trainable params: 9,507,501
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(bow_train, y_train, epochs=10, batch_size=50)
loss, accuracy = model.evaluate(bow_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0063 - accuracy: 0.9952
Epoch 2/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0061 - accuracy: 0.9947
Epoch 3/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9949
Epoch 4/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0062 - accuracy: 0.9956
Epoch 5/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9954
Epoch 6/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0063 - accuracy: 0.9952
Epoch 7/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9949
Epoch 8/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0065 - accuracy: 0.9954
Epoch 9/10
6090/6090 [==============================] - 20s 3ms/step - loss: 0.0064 - accuracy: 0.9952
Epoch 10/10
6090/6090 [==============================] - 20s 3ms/step - l

In [0]:
loss, accuracy = model.evaluate(bow_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7663
